# Análise de Dados: Data Cleaning com a base Pima Indians Diabetes

Este notebook tem como objetivo ensinar as etapas de limpeza de dados (Data Cleaning) utilizando a base de dados `pima-indians-diabetes.data.csv`.

## Pima Indians Diabetes

A base de dados **Pima Indians Diabetes** é um conjunto de dados clássico da área de saúde, frequentemente utilizado em estudos estatísticos e científicos para análise exploratória e aprendizado de máquina. Seu objetivo é estudar fatores associados ao diagnóstico de diabetes tipo 2 em mulheres de origem indígena Pima, que vivem no Arizona (EUA).

---

### Origem da Base

A base foi disponibilizada pelo **National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK)** e tornou-se amplamente conhecida por meio do repositório do **UCI Machine Learning Repository**:

🔗 https://archive.ics.uci.edu/ml/datasets/pima+indians+diabetes

---

### Descrição das Variáveis

A base contém **768 observações** com **8 variáveis preditoras** e **1 variável alvo**, todas relacionadas a exames médicos ou condições de saúde. Todas as pacientes são **mulheres com 21 anos ou mais**.

| Variável                | Tradução                           | Descrição                                                                 |
|------------------------|------------------------------------|---------------------------------------------------------------------------|
| `Pregnancies`          | Gravidezes                         | Número de vezes que a paciente esteve grávida                            |
| `Glucose`              | Glicose                            | Concentração de glicose no plasma em jejum                               |
| `BloodPressure`        | Pressão Arterial                   | Pressão arterial diastólica (mm Hg)                                      |
| `SkinThickness`        | Espessura da Pele                  | Espessura da dobra cutânea do tríceps (mm)                               |
| `Insulin`              | Insulina                           | Nível de insulina sérica em 2h após refeição (mu U/ml)                   |
| `BMI`                  | IMC                                | Índice de massa corporal (peso em kg / altura² em m²)                    |
| `DiabetesPedigreeFunction` | Histórico Familiar             | Função que expressa a probabilidade de diabetes com base no histórico familiar |
| `Age`                  | Idade                              | Idade da paciente (anos)                                                 |
| `Outcome`              | Diagnóstico                        | 0 = Não diabética, 1 = Diabética                                         |

---

### Observações importantes

- Alguns valores **zero** nas variáveis `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin` e `BMI` são considerados **inconsistências ou dados ausentes**, pois clinicamente não fazem sentido.
- A variável `Outcome` é binária e representa a **presença (1)** ou **ausência (0)** de diabetes tipo 2.

---

Esta base é amplamente usada para praticar **data cleaning**, análise descritiva e exploração de relações entre variáveis de saúde.


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carregamento dos dados

column_names = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
]
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv", names=column_names, header=0)
df.head()

## 1. Verificando a estrutura dos dados

In [ ]:
df.info()

### **Quantos atributos (variáveis) e quantas entradas o nosso conjunto de dados possui? Quais os tipos das variáveis?**

In [ ]:
print(f'O dataset possui {df.shape[0]} entradas (registros) e {df.shape[1]} atributos (variáveis).')

# Tipos de dados dos atributos
print(f'\nOs tipos dos atributos são do tipo:\n{df.dtypes}')

### **Qual a porcentagem de valores ausentes no *dataset*?**

In [ ]:
print(f'As variáveis com mais dados faltantes, em porcentagem (%), são:\n')
((df.isnull().sum() / df.shape[0])*100).sort_values(ascending=False)

### **Qual o tipo de distribuição das variáveis?**

In [ ]:
df.hist(bins=15, figsize=(20,16));

## 2. Estatísticas descritivas iniciais

In [ ]:
df.describe()

## 3. Verificando valores ausentes ou inválidos
Na base, alguns campos têm o valor **0** que pode indicar ausência de medição (por exemplo, pressão arterial = 0). Vamos verificar esses casos.

In [ ]:
cols_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in cols_with_zeros:
    print(f"{col} - Total de zeros: { (df[col] == 0).sum() }")

In [ ]:
# Substituindo zeros por NaN e contando os valores ausentes
df[cols_with_zeros] = df[cols_with_zeros].replace(0, np.nan)
df.isnull().sum()

## 4. Imputação de valores ausentes com a mediana (Usar com atenção)

In [ ]:
df.fillna(df.median(), inplace=True)
df.isnull().sum()

#Remover dados nulos
#df.dropna(inplace=True)

## 6. Visualizando distribuições e outliers

In [ ]:
plt.figure(figsize=(12, 8))
df.boxplot()
plt.title("Boxplots das variáveis com valores ausentes tratados")
plt.show()

## O que é IQR (Interquartile Range)?

O **IQR**, ou **Intervalo Interquartil**, é uma medida estatística de dispersão que descreve a faixa central de um conjunto de dados. Ele é útil especialmente para detectar **valores discrepantes (outliers)**, pois é **resistente a valores extremos**.

O IQR é calculado como a diferença entre o terceiro quartil (Q3) e o primeiro quartil (Q1):

\[
**IQR** = Q3 - Q1
\]

- **Q1 (1º Quartil)**: 25% dos dados estão abaixo desse valor.
- **Q3 (3º Quartil)**: 75% dos dados estão abaixo desse valor.
- **IQR** representa os 50% centrais dos dados.

---

## Como interpretar o IQR?

O IQR serve como base para identificar possíveis **outliers**. Valores são considerados outliers se estiverem **muito abaixo de Q1 ou muito acima de Q3**, fora do intervalo definido por:

- **Limite Inferior**:  
  \[
  Q1 - 1.5 * IQR
  \]
- **Limite Superior**:  
  \[
  Q3 + 1.5 * IQR
  \]

Qualquer dado que esteja **fora desses limites** é considerado um possível outlier.

---

## Vantagens do uso do IQR

- **Não depende da distribuição dos dados** (diferente do uso de média e desvio padrão).
- **Menos sensível a valores extremos**, tornando-o mais robusto.


In [ ]:
def detectar_outliers_iqr(coluna):
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers = df[(df[coluna] < limite_inferior) | (df[coluna] > limite_superior)]
    return outliers

outliers_insulin = detectar_outliers_iqr('Insulin')

print(outliers_insulin)

#remover outliers
#df = df.drop(outliers_insulin.index)

# Questão

- Devo remover os outliers?